This notebook performs data augmentation on the high quality data. It translates the intent into spanish and back into english.

#### INSTALL DEPENDENCIES

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=5158e01b837a1cc8a766aedc119b983dfb3c3002b6a9fa79ae4d8784bf7730ef
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce

#### IMPORT LIBRARIES

In [2]:
!git clone https://github.com/nelson-nsc/COMP0087-NLP-project.git

Cloning into 'ucl-nlp'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 173 (delta 21), reused 28 (delta 17), pack-reused 137
Receiving objects: 100% (173/173), 7.49 MiB | 12.90 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [3]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

#### DATA AUGMENTATION

In [5]:
## prepare data to augmented
df = pd.read_csv('/content/COMP0087-NLP-project/data/conala-hq-marian/hq_train.csv')
intent = df['intent'].values.tolist()

# define a method for adding special token to a batch of text
format_text = lambda code, txts: [f'>>{code}<< {txt}' for txt in txts]


In [6]:
## define the two models

# define english to spanish model
name1 = 'Helsinki-NLP/opus-mt-en-es'
eng2esp_tkn = MarianTokenizer.from_pretrained(name1)
eng2esp_model = MarianMTModel.from_pretrained(name1).to('cuda')  # model

# define spanish to english mdoel
name2 = 'Helsinki-NLP/opus-mt-es-en'
esp2eng_tkn = MarianTokenizer.from_pretrained(name2)   # tokenizer
esp2eng_model = MarianMTModel.from_pretrained(name2).to('cuda')  # model

In [7]:
## define translate method
def translate(txts, model, tokenizer, to_language):
    # prepare the text data into appropriate format
    formated_texts = format_text(to_language, txts)
    
    # translate using model
    translated = model.generate(**tokenizer(formated_texts, return_tensors="pt", padding=True).to('cuda'))

    # convert the generated tokens indices back into text
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    
    return translated_texts


In [8]:
## define method to perform back translation
def back_translate(txts):
  # english to spanish
  spanish_txts = translate(txts, eng2esp_model, eng2esp_tkn, 'es')

  # spanish back to english
  english_txts = translate(spanish_txts, esp2eng_model, esp2eng_tkn, 'en')

  return english_txts


In [9]:
## perform the augmentation
intent_aug = [back_translate([txt])[0].lower() for txt in intent]

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
lst1 = intent_aug
lst2 = df['snippet'].values.tolist()

aug_intent_df = pd.DataFrame(list(zip(lst1, lst2)), columns =['intent', 'snippet'])

In [ ]:
# save the augmented data
aug_intent_df.to_csv('/content/drive/MyDrive/coNaLa-data/train_aug.csv', index=False)